In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

# 서비스지역에 존재하는 cctv 목록

In [80]:
df = pd.read_csv('../data/tabular/CCTV_서울지역구경계_join.csv', encoding='cp949')

# 서울에 위치하는 cctv만
df = df[df['distance'] == 0]

target_district = ['동작구', '강남구', '서초구']
# 서비스 지역의 cctv만
service_df = df[df['join_SGG_NM'].isin(target_district)]

# 필요한 정보만
col = list(service_df.columns)
col = col[:6]

service_df = service_df[col]
service_df.to_csv('../data/tabular/서비스지역_CCTV_정보.csv', encoding='cp949', index=False)
display(service_df)

,RN,CCTVID,CCTVNAME,CENTERNAME,XCOORD,YCOORD
368,369,E910054,[경부선]금현동,국가교통정보센터(고속도로),127.067802,37.432619
398,399,E910084,[경부선]달래내2,국가교통정보센터(고속도로),127.061944,37.438056
454,455,E910140,[경부선]상적교,국가교통정보센터(고속도로),127.056390,37.445560
461,462,E910146,[경부선]서초,국가교통정보센터(고속도로),127.025830,37.483060
518,519,E910204,[경부선]양재,국가교통정보센터(고속도로),127.042004,37.461626
...,...,...,...,...,...,...
6796,6797,L010304,포스코,서울교통정보센터,127.056750,37.507220
6798,6799,L010152,학여울역,서울교통정보센터,127.070300,37.496410
6799,6800,L010181,한강대교남단,서울교통정보센터,126.955520,37.513460
6803,6804,L010123,한남대교남단,서울교통정보센터,127.016650,37.523670


# 서비스 지역 실시간 영상 링크 가져오기

In [101]:
service_df = pd.read_csv('../data/tabular/서비스지역_CCTV_정보.csv', encoding='cp949')
service_df.head()

,RN,CCTVID,CCTVNAME,CENTERNAME,XCOORD,YCOORD,video
0,369,E910054,[경부선]금현동,국가교통정보센터(고속도로),127.067802,37.432619,http://cctvsec.ktict.co.kr/2431/xrzNhtLoHeuKSy...
1,399,E910084,[경부선]달래내2,국가교통정보센터(고속도로),127.061944,37.438056,http://cctvsec.ktict.co.kr/501/qAbMtECOYhWx1Xy...
2,455,E910140,[경부선]상적교,국가교통정보센터(고속도로),127.056390,37.445560,http://cctvsec.ktict.co.kr/95/aKkfNVks7IVRL9Gj...
3,462,E910146,[경부선]서초,국가교통정보센터(고속도로),127.025830,37.483060,http://cctvsec.ktict.co.kr/99/h5m1fEJcpyDarNr3...
4,519,E910204,[경부선]양재,국가교통정보센터(고속도로),127.042004,37.461626,http://cctvsec.ktict.co.kr/100/FSR7QQJMppLMgPQ...


In [62]:
key = input()
url = 'https://www.utic.go.kr/guide/cctvOpenData.do?key={}'.format(key)
xpath='//*[@id="vid_html5_api"]/source'
xpath2='//*[@id="innerObject"]/div/script'
m3u8_url = {}

chromedriver_path = '../chromedriver/chromedriver'
socket.setdefaulttimeout(30)
driver = webdriver.Chrome(chromedriver_path)
driver.get(url)
driver.maximize_window()

click_elements = []
trs = driver.find_elements(By.TAG_NAME, 'tr')

for tr in tqdm(trs) :
    # 첫번째 anchor요소만 a_tags에 저장
    cctv, management = tr.find_elements(By.TAG_NAME, 'a')
#     print(cctv.get_attribute('innerText'))
    management = management.get_attribute('innerText')
    
    if management=='국가교통정보센터(고속도로)' or management=='서울교통정보센터' :
        cctv_name = cctv.get_attribute('innerText').split(".")[-1]
        if cctv_name in list(service_df['CCTVNAME']) :
            print(cctv_name)
            click_elements.append([cctv_name, cctv])
    
print('anchor tag 저장 완료')

for cctv_name, element in tqdm(click_elements) :
    element.click()
    time.sleep(3)
    driver.switch_to.window(driver.window_handles[1])
    try :
        # 서울시 
        src = driver.find_element('xpath', xpath).get_attribute('src')
        m3u8_url[cctv_name] = src
    except Exception as e:
        try :
            # 국가교통정보센터
            src = driver.find_element('xpath', xpath2).get_attribute('innerText')
            src = src.split("'")[3]
            m3u8_url[cctv_name] = src
        except : 
            print('{} fail'.format(cctv_name))
    # popup close
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    
driver.quit()

/var/folders/63/8nyxhd_90vg4rctvs_pfm8440000gn/T/ipykernel_17697/2270164647.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver_path)


  0%|          | 0/7873 [00:00<?, ?it/s]

[경부선]금현동
[경부선]달래내2
[경부선]상적교
[경부선]서초
[경부선]양재
[경부선]원지동
[용인서울선]세곡교
[용인서울선]세곡터널입구서울
[용인서울선]세곡터널입구용인
[용인서울선]세곡터널출구서울
[용인서울선]헌릉IC
AT센터
강남세브란스
강남역
강남터미널
경기고
경남아파트
교보타워R
교육개발원입구
구룡터널
구반포R
국립국악원
국립현충원
내곡IC
노량진삼거리
논현역
대치사거리
도산공원
동호대교남단
르네상스
매봉터널
반포IC
반포대교남단
봉은교서단
봉은사삼거리
사당역
삼릉공원
삼성역
삼정
상도
서울성모병원
서울세관사거리
서초IC
서초역사거리
선암IC
성수대교
성수대교남단
세곡사거리
수서IC
수서역
압구정R
양재IC
양재역
여의교남단
역삼역
염곡
염곡IC
영동대교남단
영동전화국
원불교
은마아파트
을지병원R
이수교
이수역
일원터널
잠원IC
장승배기
청담공원
청담교
청담사거리
코엑스
탄천1교
탄천2교
포스코
학여울역
한강대교남단
한남대교남단
한남대교남단2
anchor tag 저장 완료


  0%|          | 0/78 [00:00<?, ?it/s]

In [82]:
service_df['video'] = None

for key, item in m3u8_url.items() :
    service_df.loc[df['CCTVNAME']==key, 'video'] = item

service_df.to_csv('../data/tabular/서비스지역_CCTV_정보.csv', encoding='cp949', index=False)
service_df

,RN,CCTVID,CCTVNAME,CENTERNAME,XCOORD,YCOORD,video
368,369,E910054,[경부선]금현동,국가교통정보센터(고속도로),127.067802,37.432619,http://cctvsec.ktict.co.kr/2431/xrzNhtLoHeuKSy...
398,399,E910084,[경부선]달래내2,국가교통정보센터(고속도로),127.061944,37.438056,http://cctvsec.ktict.co.kr/501/qAbMtECOYhWx1Xy...
454,455,E910140,[경부선]상적교,국가교통정보센터(고속도로),127.056390,37.445560,http://cctvsec.ktict.co.kr/95/aKkfNVks7IVRL9Gj...
461,462,E910146,[경부선]서초,국가교통정보센터(고속도로),127.025830,37.483060,http://cctvsec.ktict.co.kr/99/h5m1fEJcpyDarNr3...
518,519,E910204,[경부선]양재,국가교통정보센터(고속도로),127.042004,37.461626,http://cctvsec.ktict.co.kr/100/FSR7QQJMppLMgPQ...
...,...,...,...,...,...,...,...
6796,6797,L010304,포스코,서울교통정보센터,127.056750,37.507220,http://210.179.218.53:1935/live/564.stream/pla...
6798,6799,L010152,학여울역,서울교통정보센터,127.070300,37.496410,http://210.179.218.52:1935/live/175.stream/pla...
6799,6800,L010181,한강대교남단,서울교통정보센터,126.955520,37.513460,http://210.179.218.51:1935/live/237.stream/pla...
6803,6804,L010123,한남대교남단,서울교통정보센터,127.016650,37.523670,http://210.179.218.52:1935/live/146.stream/pla...


In [102]:
# shell 반복문 실행을 위한 텍스트 파일 생성
df = service_df[service_df['CENTERNAME']=='서울교통정보센터']
link_name = list(df['video']+'-'+df['CCTVID']+'.mp4\n')

with open('../data/link_name.txt', 'w') as file :
    file.writelines(link_name)